## Import Libraries

In [11]:
import time
import rospy
import numpy as np
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from dual_quaternion import Quaternion
from dual_quaternion import DualQuaternion
from casadi import Function

## Working with Symbolic Variables

In [12]:
# Defining First transformation
theta_1 = ca.MX.sym('theta_1', 1, 1)
theta_1p = ca.MX.sym('theta_1p', 1, 1)
n_1 = ca.MX([0.0, 0.0, 1.0])
q_1 = ca.vertcat(ca.cos(theta_1/2), ca.sin(theta_1/2)@n_1)
tx_1 = ca.MX.sym("x_1", 1, 1)
t_1 = ca.MX([0.0, 0.0, 0.0, 0.0])

theta_2 = ca.MX.sym('theta_2', 1, 1)
theta_2p = ca.MX.sym('theta_2p', 1, 1)
n_2 = ca.MX([0.0, 0.0, 1.0])
q_2 = ca.vertcat(ca.cos(theta_2/2), ca.sin(theta_2/2)@n_2)
tx_2 = ca.MX.sym("x_2", 1, 1)
t_2 = ca.vertcat(0.0, tx_2, 0.0, 0.0)


aux_symbolic_total = ca.vertcat(theta_1, theta_2, tx_2)
symbolic_actions = ca.vertcat(theta_1, theta_2)
control = ca.vertcat(theta_1p, theta_2p)

### Computing Sequential Transformations

In [13]:
Q1 = DualQuaternion.from_pose(quat = q_1, trans = t_1)
Q2 = DualQuaternion.from_pose(quat = q_2, trans = t_2)
Q3 = Q1 * Q2

## Generating Function Using CasADi

In [14]:
Function_DualQuat_total = Function('DualQuat_total', [aux_symbolic_total], [Q3.get[:, 0]])
Function_DualQuat_total_trans = Function('DualQuat_total_trans', [aux_symbolic_total], [Q3.get_trans.get[:, 0]])
Function_DualQuat_total_quat = Function('DualQuat_total_quat', [aux_symbolic_total], [Q3.get_quat.get[:, 0]])

## Using Functions

In [15]:
input_t = np.array([4.97, 0.0, 1])
trans = Function_DualQuat_total_trans(input_t)
trans

DM([-0, 0.254771, -0.967001, 0])

### Calculating Jacobians

In [16]:
J = ca.jacobian(Q3.get[:, 0], symbolic_actions)

In [17]:
Function_J = Function("Function_J", [aux_symbolic_total], [J])

In [18]:
input_t = np.array([-3.15, 0.0, 1])
Jacobian = Function_J(input_t)
print(Jacobian)


[[0.499996, 0.499996], 
 [0, 0], 
 [0, 0], 
 [-0.00210183, -0.00210183], 
 [-0, 0], 
 [0.249998, -0.249998], 
 [-0.00105092, 0.00105092], 
 [0, 0]]


## Computing Differential Kinematics

In [19]:
diff_kinematics = J @ control

Function_diff_kinematics = Function("Function_diff_kinematics", [aux_symbolic_total, control], [diff_kinematics])

In [20]:
input_t = np.array([-3.15, 0.0, 1])
input_t_p = np.array([0.36, 0.0])
Xp = Function_diff_kinematics(input_t, input_t_p)
Xp

DM([0.179998, 0, 0, -0.000756659, 0, 0.0899992, -0.000378329, 0])